# EEG-ERP Preprocessing
---
## Learning Objectives
- Be able to describe the standard steps in preprocessing EEG data for ERP analysis, including filtering, visual inspection, automated artifact detection and removal using independent components analysis, bad channel removal and interpolation, re-referencing, and averaging
- Be able to describe the motivation for each of these preprocessing steps
- Be able to perform each preprocessing step using MNE

---

## Introduction
**Preprocessing** is a series of signal processing steps that are performed on data prior to analysis (EDA and/or statistical analysis) and interpretation. In virtually all forms of neuroimaging data, including EEG and MEG, preprocessing is necessary in order to remove noise and obtain a clean signal of interest. In the case of EEG, the data recorded from the scalp are inevitably a mixture of signals generated from the brain (which we likely care about), and other signals generated by sources other than the brain (which we generally don't wish to interpret). In EEG, sources of noise can be physiological and non-physiological.

**Physiological** sources of noise include muscles (especially face and neck muscles, which generate electrical potentials when contracting and relaxing), the eyes (which create distinctive signals both during blinks, and when the eyeballs move), and sometimes heartbeats (the heart produces strong, distinctive electrical signals as commonly measured with the electrocardiogram, or EKG) and breathing; the latter two sources typically are more of a problem for MEG than EEG data.

**Non-physiological** sources include artifacts generated when electrodes move relative to the scalp, when the electrode wires move, and ambient electromagnetic noise. Electromagnetic noise is created by virtually any device that uses electricity (except some battery-powered devices). In North America and some other parts of the world, **line voltage** — the alternating current that is provided through wired electrical outlets — oscillates at 60 Hz; in Europe and other parts of the world, line voltage alternates at 50 Hz. Thus depending on where the EEG data were recorded, they will likely contain a fairly distinct peak in the frequency spectrum at 50 or 60 Hz. While line noise is the most common source of electromagnetic interference, noise at other frequencies can also be produced by other equipment near the neuroimaging system (though sometimes this may be in another room or another floor of the building, and difficult to identify), or even stronger, transient sources (such as the 2-way radio of a passing truck).

Some people, when they first learn about preprocessing, think it sounds a lot like cheating — "doctoring" the signal to make what you want appear out of it. This is not really the case, however. Preprocessing steps are carefully designed and implemented based on our understanding of the source and nature of particular sources of noise, as well as our understanding about the nature of the electrical activity generated by the brain. Because the skull is such a poor conductor of electricity, the amplitude of EEG signals recorded from the scalp is very small. In contrast, the amplitude of signals from noise sources — especially from the muscles and eyes — are often an order of magnitude or more larger. Thus if we fail to remove these known sources of noise from the data, it would be unlikely that we would detect the brain activity at all, and at best our ability to do so and make valid interpretations would be greatly diminished.


In this lesson we will describe each standard step in preprocessing EEG data for ERP analysis, including why it is done, and how, using the MNE package.

---

## Load packages

In [2]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import mne

# from mne.viz.ica import _prepare_data_ica_properties
# from scipy.stats import zscore
# from autoreject import Ransac, get_rejection_threshold, AutoReject
# import os

## Import raw data

For EEG, raw data is typically stored in a single file containing continuous EEG data. There are many different file formats for EEG data; most of these are developed by the manufacturer or a particular EEG system. Fortunately, MNE provides functions to import data from most common EEG systems. 

In the present case, we are working with data from a system sold by Brain Products, whose software is called *Brain Vision*. So we will use MNE's `read_raw_brainvision()` function. As described previously, many of MNE's functions are nested inside subfolders (modules); for example, all of the file input-output (I/O) routines are in the module `mne.io`. So when we call the `read_raw_brainvision()` function, we have to do so as `mne.io.read_raw_brainvision()`. The one required argument for this function is the file name, and that's all we need for now: 

In [3]:
raw_file = 'data/P4.vhdr'

raw = mne.io.read_raw_brainvision(raw_file)

Extracting parameters from data/P4.vhdr...
Setting channel info structure...


/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39870/3013224364.py:3: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(raw_file)


## Filtering